In [1]:
import os
import pandas as pd
import requests
import bs4
import lxml
import functions.web_scrapper_functions as scrpr
import mysql.connector
import json
import datetime

import logging
import warnings
warnings.filterwarnings("ignore")

logging.basicConfig(filename='Parfum AE Web Scrapper Log.log', level=logging.INFO,
format='%(asctime)s:%(levelname)s:%(message)s')

In [2]:
frag_type_dict = scrpr.get_frag_types()
print("Available fragrance types: \n", frag_type_dict.keys())

Available fragrance types: 
 dict_keys(['Body Mist', 'Car Perfume Diffuser', 'Perfume Oils', 'Our Creations', 'Dahn Al Oud', 'Interior Perfume', 'Musk Oil', 'Perfume Reed Diffuser'])


In [3]:
prod_list_dict = {}
start_time = datetime.datetime.now()

for key, val in frag_type_dict.items():
    try:
        logging.info('Currently scrapping ', key)
        prod_list_dict[key] = scrpr.prod_list(val)
        logging.info('Number of products: ', len(prod_list_dict[key]))
    except:
        logging.error("Error in ", key)
end_time = datetime.datetime.now()
time_taken = end_time - start_time
logging.info(f"Time taken to retrieve list of all products: {time_taken}")

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/chucky/Documents/Projects/Project 4 - PARFUM.AE/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/chucky/Documents/Projects/Project 4 - PARFUM.AE/venv/lib/python3

In [4]:
start_time = datetime.datetime.now()
products = scrpr.prod_info(prod_list_dict)
end_time = datetime.datetime.now()
time_taken = end_time - start_time
logging.info(f"Time taken to retrieve information for each product: {time_taken}")

In [5]:
products

[[None, 'Plum Perfect Body Mist 150ml', 5, 50.0, 6, 50.0, False, []],
 [None, 'Triumph Body Mist 150ml', 5, 80.0, 2, 80.0, False, []],
 [None, 'Falcon Musk Body Mist 150ml', 5, 70.0, 1, 70.0, False, []],
 [None, 'Lavender Dahyana Body Mist 150ml', 5, 60.0, 1, 60.0, False, []],
 [None, 'Washington Apple Body Mist 150ml', 5, 70.0, 1, 70.0, False, []],
 [None, 'Apotheosis Body Mist 150ml', 0, 80.0, 0, 80.0, False, []],
 [None, 'Cedar Breeze Body Mist 150ml', 0, 70.0, 0, 70.0, False, []],
 [None, 'Violet Amber Body Mist 150ml', 0, 60.0, 0, 60.0, False, []],
 [None, 'Citrus Sweet Body Mist 150ml', 0, 60.0, 0, 60.0, False, []],
 [None, 'Rich Princessa Body Mist 150ml', 0, 70.0, 0, 70.0, False, []],
 [None, 'Windy Musk Body Mist 150ml', 0, 50.0, 0, 50.0, False, []],
 [None, 'Silk Roses Body Mist 150ml', 0, 75.0, 0, 75.0, False, []],
 [None, 'Misty Mystery Body Mist 150ml', 0, 70.0, 0, 70.0, False, []],
 [None, 'Amber Temptation Body Mist 150ml', 0, 70.0, 0, 70.0, False, []],
 [None, 'Poisonou

In [34]:
reviews

{'Sabrina ': 'Actually I like it this product good job team Parfum.ae',
 'Abden': 'Good Product',
 'noor  Almazrouei': 'Really its very good and good for summer! ',
 'liyaan': "This is exactly what I was looking for. Crisp, clean, woody, bitter and medicinal with the slightly cool mintiness. This has set the bar for me personally and is the scent of kyara I\\'ve heard described and knew I had to try. development on the skin is divine. Probably best not to use before bed, it\\'s strong! No idea how the price is so reasonable, baffles me. I know I\\'ll be searching for a long time for something cultivated and dirt cheap that smells like this so I can slather it on daily. ",
 'Rufat Abdullayev': 'A fabulous Oud oil, thick like honey, among the best agarwoody perfumes '}

In [36]:
product_reviews('https://www.parfum.ae/oud-cambodi-synthetic/')

[{'Sabrina ': 'Actually I like it this product good job team Parfum.ae'},
 {'Abden': 'Good Product'},
 {'noor  Almazrouei': 'Really its very good and good for summer! '},
 {'liyaan': "This is exactly what I was looking for. Crisp, clean, woody, bitter and medicinal with the slightly cool mintiness. This has set the bar for me personally and is the scent of kyara I\\'ve heard described and knew I had to try. development on the skin is divine. Probably best not to use before bed, it\\'s strong! No idea how the price is so reasonable, baffles me. I know I\\'ll be searching for a long time for something cultivated and dirt cheap that smells like this so I can slather it on daily. "},
 {'Rufat Abdullayev': 'A fabulous Oud oil, thick like honey, among the best agarwoody perfumes '}]

In [ ]:
conn = mysql.connector.connect(
    host = "localhost",
    user = os.environ.get("MYSQL_USERNAME"),
    password = os.environ.get("MYSQL_PASSWORD"),
    database = "parfumae_products"
)

cursor = conn.cursor()

In [ ]:

try:
    for key, val in frag_type_dict.items():
        serialized_value = json.dumps(val)

        insert_query = "INSERT INTO fragrance_types (name, url) VALUES (%s, %s)"
        cursor.execute(insert_query, (key, serialized_value))
    
    conn.commit()

except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()

cursor.close()
conn.close()